In [49]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud


locations = pd.read_csv("./data/table_10_incidents_bias_motivation_by_location_2016.csv")

locationsIncidents = locations.iloc[2::,0:2]
locationsIncidents.columns = ['words', 'counts']
tuples = [tuple(x) for x in locationsIncidents.values]
wordcloud = WordCloud()


AttributeError: 'list' object has no attribute 'items'

In [50]:
wordcloud = WordCloud()

wordcloud.generate_from_frequencies(frequencies=tuples)
plt.figure(figsize=(10, 10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()



AttributeError: 'list' object has no attribute 'items'

# Problem Overview

The purpose of the project, source of the data, and any pertinent information about the topic area are introduced. You may also want to include links to the code, or information about the team.

Use info from proposal

# Data Preparation

Determine what to do about null values
Discuss variables
Continue discussion of data

# Exploratory Data Analysis

Exploration questions - how many crimes per state
High-level insights, trends, outliers, etc.


In [64]:
# Read in CSVs
#TODO: WHAT TO DO WITH NULL VALUES?
hate_crimes_by_socioeconomic = pd.read_csv("./data/hate_crimes.csv")


motivation_by_num_situation = pd.read_csv("./data/table_1_incidents_offenses_victims_and_known_offenders_by_bias_motivation_2016.csv").drop(["Unnamed: 5", "Unnamed: 6", "Unnamed: 7"], axis = 1)
offense_type_by_situation = pd.read_csv("./data/table_2_incidents_offenses_victims_and_known_offenders_by_offense_type_2016.csv").drop(["Unnamed: 5", "Unnamed: 6"], axis = 1)
offense_type_by_offender_race = pd.read_csv("./data/table_3_offenses_known_offenders_race_and_ethnicity_by_offense_type_2016.csv")
motivation_by_crime_type = pd.read_csv("./data/table_4_offenses_offense_type_by_bias_motivation_2016.csv")
motivation_by_offender_race = pd.read_csv("./data/table_5_offenses_known_offenders_race_and_ethnicity_by_bias_motivation_2016.csv").drop(["Unnamed: 14", "Unnamed: 15", " "], axis = 1)
location_by_motivation = pd.read_csv("./data/table_10_incidents_bias_motivation_by_location_2016.csv").drop(["Unnamed: 9", "Unnamed: 10", "Unnamed: 11", "Unnamed: 12"], axis = 1)
state_by_crime_type = pd.read_csv("./data/table_11_offenses_offense_type_by_participating_state_2016.csv")


# Statistical Modeling

Null Hypothesis
